In [1]:
import tensorflow as tf
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd
import random

In [2]:
import os

from tensorflow.keras import backend
from tensorflow.keras import layers
from tensorflow.keras.applications import imagenet_utils

In [3]:
class_number = 15
category_number = 13

bone_cates = ['far-phalanx1', 'far-phalanx3', 'far-phalanx5', 'metacarpal1',
       'metacarpal3', 'metacarpal5', 'middle-phalanx3', 'middle-phalanx5',
       'nearly-phalanx1', 'nearly-phalanx3', 'nearly-phalanx5', 'radius',
       'ulna']

In [4]:
def number2onehot(number):
    if type(number) != np.ndarray:
        number = np.array(number)
    return np.eye(class_number)[number].reshape((-1, category_number*class_number))

def onehot2number(res):
    return np.argmax(res.reshape((-1, category_number, class_number)), axis=-1)

In [5]:
class Dense_1(tf.keras.Model):
    def __init__(self, batch_size, bn_axis=-1):
        super(Dense_1, self).__init__()
        self.batch_size = batch_size
        self.pad1 =  layers.ZeroPadding2D(padding=((3, 3), (3, 3)), name='densenet_1_pad')
        self.conv1 =  layers.Conv2D(64, 3, strides=1, use_bias=False, name='densenet_1_conv')
        self.bn = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='densenet_1_bn')
        self.relu = x = layers.Activation('relu', name='densenet_1_relu')
        self.pad2 = layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name='densenet_1_pad_2')
        self.pool = layers.MaxPooling2D(3, strides=2, name='densenet_1_pool_1')

    def call(self, x):
        x = tf.gather(x, indices=tf.range(self.batch_size))
        x = tf.reshape(x, (-1, 256, 256, 1), name='densenet_1_reshape')
        x = self.pad1(x)
        x = self.conv1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.pad2(x)
        x = self.pool(x)
        return x

In [6]:
class Dense_conv_block(tf.keras.Model):
    def __init__(self, growth_rate, name, bn_axis=-1):
        super(Dense_conv_block, self).__init__()
        self.bn1 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')
        self.a1 = layers.Activation('relu', name=name + '_1_relu')
        self.c1 = layers.Conv2D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')
        self.bn2 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_2_bn')
        self.a2 = layers.Activation('relu', name=name + '_2_relu')
        self.c2 = layers.Conv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')
        self.concat = layers.Concatenate(axis=bn_axis, name=name + '_concat')
        
    def call(self, x):
        x1 = self.bn1(x)
        x1 = self.a1(x1)
        x1 = self.c1(x1)
        x1 = self.bn2(x1)
        x1 = self.a2(x1)
        x1 = self.c2(x1)
        x = self.concat([x, x1])
        return x

In [7]:
class Dense_transition_block(tf.keras.Model):
    def __init__(self,filters, name, bn_axis=-1):
        super(Dense_transition_block, self).__init__()
        self.bn = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')
        self.a = layers.Activation('relu', name=name + '_relu')
        self.c = layers.Conv2D(filters,1,use_bias=False,name=name + '_conv')
        self.p = layers.AveragePooling2D(2, strides=2, name=name + '_pool')
        
    def call(self, x):
        x = self.bn(x)
        x = self.a(x)
        x = self.c(x)
        x = self.p(x)
        return x

In [8]:
class Dense_block(tf.keras.Model):
    def __init__(self, blocks, name):
        super(Dense_block, self).__init__()
        self.convs = []
        self.blocks = blocks
        for i in range(blocks):
            self.convs.append(Dense_conv_block(32, name=name + '_block' + str(i + 1)))
    
    def call(self, x):
        for i in range(self.blocks):
            x = self.convs[i](x)
        return x

In [9]:
class DenseNet(tf.keras.Model):
    def __init__(self, 
        blocks,
        batch_size,
        bn_axis=-1,
        classes=category_number*class_number):
        
        super(DenseNet, self).__init__()
        self.batch_size = batch_size
        self.d1 = Dense_1(batch_size=self.batch_size)
        self.d2 = Dense_block(blocks[0], name='densenet_2')
        self.t2 = Dense_transition_block(blocks[0], name='transition_2')
        self.d3 = Dense_block(blocks[1], name='densenet_3')
        self.t3 = Dense_transition_block(blocks[1], name='transition_3')
        self.d4 = Dense_block(blocks[2], name='densenet_4')
        self.t4 = Dense_transition_block(blocks[2], name='transition_4')
        self.d5 = Dense_block(blocks[3], name='densenet_5')

        self.bn6 = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='fc_bn6')
        self.a6 = layers.Activation('relu', name='fc_relu6')
        self.avg_pool = layers.GlobalAveragePooling2D(name='fc_avg_pool')

        self.pred = layers.Dense(classes, activation='sigmoid', name='predictions')
    
    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        x = self.t2(x)
        x = self.d3(x)
        x = self.t3(x)
        x = self.d4(x)
        x = self.t4(x)
        x = self.d5(x)
        x = self.bn6(x)
        x = self.a6(x)
        x = self.avg_pool(x)
        
        x = tf.gather(x, indices=tf.range(category_number*self.batch_size))
        x = tf.reshape(x, (self.batch_size, -1))
        x = self.pred(x)
        return x 

In [11]:
# imgs: shape:  np.array (1, 13, 256, 256)
#               13 sort like bone_cates
# file_ids:  list 
# return:  df
def test(model, imgs, file_id):
    res = model.predict(imgs)
    res = onehot2number(res)
    df = pd.DataFrame(res, columns=bone_cates)
    df['file'] = file_id
    return df

In [12]:
model = DenseNet(blocks=[4, 8, 16, 16], batch_size=1)
model.load_weights('M_dense_1_63_weights.h5')